# Latency Lab Analysis

This notebook reads `results/index.csv` and shows basic comparisons.


In [ ]:
import sys
from pathlib import Path

def find_repo_root(start: Path) -> Path:
    for parent in [start] + list(start.parents):
        if (parent / "scripts" / "results_lib.py").exists():
            return parent
    return start

repo_root = find_repo_root(Path().resolve())
sys.path.append(str(repo_root / "scripts"))

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from results_lib import load_index, load_samples

sns.set_theme(style="whitegrid")


## Run benchmarks from the notebook

Use this helper to launch a benchmark and print the latest stdout summary.


In [ ]:
import sys
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display, clear_output

def find_repo_root(start: Path) -> Path:
    for parent in [start] + list(start.parents):
        if (parent / "scripts" / "notebook_runner.py").exists():
            return parent
    return start

repo_root = find_repo_root(Path().resolve())
scripts_dir = repo_root / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.append(str(scripts_dir))

import notebook_runner as nb

bench_path = repo_root / "build" / "bench"
results_dir = repo_root / "results"

try:
    cases = nb.list_cases(bench_path)
except Exception as exc:
    cases = ["noop"]
    print(f"Could not list cases: {exc}")
    print(f"Build bench at {bench_path} to populate the list.")

case_checks = {
    name: widgets.Checkbox(value=False, description=name) for name in cases
}
checkbox_box = widgets.VBox(
    list(case_checks.values()),
    layout=widgets.Layout(max_height="240px", overflow="auto"),
)
filter_text = widgets.Text(value="", description="Filter")

def apply_filter(_=None):
    query = filter_text.value.strip().lower()
    if not query:
        visible = list(case_checks.values())
    else:
        visible = [
            checkbox
            for name, checkbox in case_checks.items()
            if query in name.lower()
        ]
    checkbox_box.children = visible

filter_text.observe(apply_filter, names="value")
apply_filter()

lab = widgets.Text(value="os", description="Lab")
iters = widgets.IntText(value=10000, description="Iters")
warmup = widgets.IntText(value=1000, description="Warmup")
pin_cpu = widgets.IntText(value=-1, description="Pin CPU")
tags = widgets.Text(value="quiet", description="Tags (csv)")
update_mode = widgets.RadioButtons(
    options=[
        ("Append (default)", "append"),
        ("Skip summary/index", "skip"),
        ("Replace matching in index", "replace"),
    ],
    value="append",
    description="Update",
)

select_all = widgets.Button(description="Select all")
deselect_all = widgets.Button(description="Deselect all")
run_button = widgets.Button(description="Run selected")
output = widgets.Output()

def select_all_clicked(_):
    for checkbox in case_checks.values():
        checkbox.value = True

def deselect_all_clicked(_):
    for checkbox in case_checks.values():
        checkbox.value = False

select_all.on_click(select_all_clicked)
deselect_all.on_click(deselect_all_clicked)

def run_clicked(_):
    with output:
        clear_output()
        selected = [
            name for name, checkbox in case_checks.items() if checkbox.value
        ]
        if not selected:
            print("No cases selected.")
            return
        tag_list = [t.strip() for t in tags.value.split(",") if t.strip()]
        pin = None if pin_cpu.value < 0 else pin_cpu.value
        for idx, case_name in enumerate(selected, start=1):
            print(f"[{idx}/{len(selected)}] {case_name}")
            try:
                result = nb.run_case(
                    bench_path=bench_path,
                    lab=lab.value,
                    case=case_name,
                    results_dir=results_dir,
                    iters=iters.value,
                    warmup=warmup.value,
                    pin_cpu=pin,
                    tags=tag_list,
                    update_mode=update_mode.value,
                )
            except Exception as exc:
                print(f"Run failed for {case_name}: {exc}")
                continue
            print(f"Run dir: {result.run_dir}")
            print(result.read_stdout())

run_button.on_click(run_clicked)
display(
    widgets.VBox(
        [
            filter_text,
            widgets.HBox([select_all, deselect_all, run_button]),
            checkbox_box,
            lab,
            iters,
            warmup,
            pin_cpu,
            tags,
            update_mode,
        ]
    ),
    output,
)


In [ ]:
index = load_index(repo_root / "results" / "index.csv")

if hasattr(index, "columns") and "run_dir" in index.columns:
    def _abs_run_dir(value):
        if not isinstance(value, str) or not value:
            return value
        path = Path(value)
        if path.is_absolute():
            return value
        return str((repo_root / path).resolve())

    index["run_dir"] = index["run_dir"].apply(_abs_run_dir)

index.head()


In [ ]:
# Pick a case to compare pinned vs unpinned (adjust as needed)
case_name = index["case"].iloc[0] if len(index) else "noop"
subset = index[index["case"] == case_name]

display(subset[["case","tags","pin_cpu","p50","p95","p99","p999","mean"]])


In [ ]:
# Long-form view of quantiles for the chosen case
if len(subset):
    plot_df = subset.melt(
        id_vars=["tags","pin_cpu"],
        value_vars=["p50","p95","p99","p999"],
        var_name="quantile",
        value_name="ns",
    )
    plt.figure(figsize=(8,4))
    sns.barplot(data=plot_df, x="quantile", y="ns", hue="pin_cpu")
    plt.title(f"{case_name} quantiles by pin_cpu")
    plt.show()


In [ ]:
# ECDF of raw samples for the first two runs (if raw data is present)
def short_label(value):
    if not isinstance(value, str):
        return str(value)
    path = Path(value)
    parts = path.parts
    if "results" in parts:
        idx = parts.index("results")
        tail = parts[idx + 1 :]
        if len(tail) >= 3:
            return "/".join(tail[:3])
        if tail:
            return "/".join(tail)
    try:
        return str(path.relative_to(repo_root))
    except Exception:
        return path.name or str(value)

if len(index) >= 2:
    run_a = index.iloc[0]["run_dir"]
    run_b = index.iloc[1]["run_dir"]
    samples_a = load_samples(run_a)
    samples_b = load_samples(run_b)

    plt.figure(figsize=(12,8))
    sns.ecdfplot(samples_a, label=short_label(run_a))
    sns.ecdfplot(samples_b, label=short_label(run_b))
    plt.xlabel("ns")
    plt.ylabel("ECDF")
    plt.legend()
    plt.title("ECDF comparison")
    plt.show()
